# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import warnings
import math
import matplotlib.ticker as ticker
warnings.filterwarnings('ignore') #ignora os avisos


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import tree 

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import plot_roc_curve

# Importação dos dados

In [ ]:
url = 'https://raw.githubusercontent.com/renanmath/Bootcamp_Projeto_Final/main/Dados/dados_resumidos_UTI'

In [ ]:
dados_resumidos = pd.read_csv(url)
dados_resumidos.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,CALCIUM_MEDIAN,CALCIUM_MEAN,CALCIUM_MIN,CALCIUM_MAX,GLUCOSE_MEDIAN,GLUCOSE_MEAN,GLUCOSE_MIN,GLUCOSE_MAX,HEMATOCRITE_MEDIAN,HEMATOCRITE_MEAN,HEMATOCRITE_MIN,HEMATOCRITE_MAX,HEMOGLOBIN_MEDIAN,HEMOGLOBIN_MEAN,HEMOGLOBIN_MIN,HEMOGLOBIN_MAX,LACTATE_MEDIAN,LACTATE_MEAN,LACTATE_MIN,LACTATE_MAX,LEUKOCYTES_MEDIAN,LEUKOCYTES_MEAN,LEUKOCYTES_MIN,...,POTASSIUM_MEAN,POTASSIUM_MIN,POTASSIUM_MAX,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_MEAN,SAT02_VENOUS_MIN,SAT02_VENOUS_MAX,SODIUM_MEDIAN,SODIUM_MEAN,SODIUM_MIN,SODIUM_MAX,UREA_MEDIAN,UREA_MEAN,UREA_MIN,UREA_MAX,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,WINDOW,ICU
0,0,1,0.6,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.183673,0.183673,0.183673,0.183673,-0.891993,-0.891993,-0.891993,-0.891993,0.090147,0.090147,0.090147,0.090147,0.109756,0.109756,0.109756,0.109756,1.000000,1.000000,1.000000,1.000000,-0.835844,-0.835844,-0.835844,...,-0.518519,-0.518519,-0.518519,0.345679,0.345679,0.345679,0.345679,-0.028571,-0.028571,-0.028571,-0.028571,-0.836145,-0.836145,-0.836145,-0.836145,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,0-2,1
2,2,0,0.1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.780261,-0.780261,-0.780261,-0.780261,0.144654,0.144654,0.144654,0.144654,0.158537,0.158537,0.158537,0.158537,1.000000,1.000000,1.000000,1.000000,-0.382773,-0.382773,-0.382773,...,-0.703704,-0.703704,-0.703704,0.345679,0.345679,0.345679,0.345679,0.085714,0.085714,0.085714,0.085714,-0.836145,-0.836145,-0.836145,-0.836145,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,0-2,1
3,3,0,0.4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.326531,0.326531,0.326531,0.326531,-0.891993,-0.891993,-0.891993,-0.891993,-0.203354,-0.203354,-0.203354,-0.203354,-0.219512,-0.219512,-0.219512,-0.219512,-0.828421,-0.828421,-0.828421,-0.828421,-0.729239,-0.729239,-0.729239,...,-0.777778,-0.777778,-0.777778,0.580247,0.580247,0.580247,0.580247,0.200000,0.200000,0.200000,0.200000,-0.937349,-0.937349,-0.937349,-0.937349,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,0-2,0
4,4,0,0.1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.851024,-0.851024,-0.851024,-0.851024,0.358491,0.358491,0.358491,0.358491,0.304878,0.304878,0.304878,0.304878,1.000000,1.000000,1.000000,1.000000,-0.702202,-0.702202,-0.702202,...,-0.592593,-0.592593,-0.592593,0.345679,0.345679,0.345679,0.345679,0.142857,0.142857,0.142857,0.142857,-0.903614,-0.903614,-0.903614,-0.903614,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,0.333333,-0.153846,0.160377,-0.586207,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,-0.076923,-0.351351

In [ ]:
dados_limpos = dados_resumidos.drop(['PATIENT_VISIT_IDENTIFIER', 'WINDOW', 'AGE_ABOVE65', 'GENDER'], axis = 1)
dados_limpos.head()

,AGE_PERCENTIL,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,CALCIUM_MEDIAN,CALCIUM_MEAN,CALCIUM_MIN,CALCIUM_MAX,GLUCOSE_MEDIAN,GLUCOSE_MEAN,GLUCOSE_MIN,GLUCOSE_MAX,HEMATOCRITE_MEDIAN,HEMATOCRITE_MEAN,HEMATOCRITE_MIN,HEMATOCRITE_MAX,HEMOGLOBIN_MEDIAN,HEMOGLOBIN_MEAN,HEMOGLOBIN_MIN,HEMOGLOBIN_MAX,LACTATE_MEDIAN,LACTATE_MEAN,LACTATE_MIN,LACTATE_MAX,LEUKOCYTES_MEDIAN,LEUKOCYTES_MEAN,LEUKOCYTES_MIN,LEUKOCYTES_MAX,LINFOCITOS_MEDIAN,LINFOCITOS_MEAN,...,POTASSIUM_MEDIAN,POTASSIUM_MEAN,POTASSIUM_MIN,POTASSIUM_MAX,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_MEAN,SAT02_VENOUS_MIN,SAT02_VENOUS_MAX,SODIUM_MEDIAN,SODIUM_MEAN,SODIUM_MIN,SODIUM_MAX,UREA_MEDIAN,UREA_MEAN,UREA_MIN,UREA_MAX,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,ICU
0,0.6,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.183673,0.183673,0.183673,0.183673,-0.891993,-0.891993,-0.891993,-0.891993,0.090147,0.090147,0.090147,0.090147,0.109756,0.109756,0.109756,0.109756,1.000000,1.000000,1.000000,1.000000,-0.835844,-0.835844,-0.835844,-0.835844,-0.914938,-0.914938,...,-0.518519,-0.518519,-0.518519,-0.518519,0.345679,0.345679,0.345679,0.345679,-0.028571,-0.028571,-0.028571,-0.028571,-0.836145,-0.836145,-0.836145,-0.836145,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,1
2,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.780261,-0.780261,-0.780261,-0.780261,0.144654,0.144654,0.144654,0.144654,0.158537,0.158537,0.158537,0.158537,1.000000,1.000000,1.000000,1.000000,-0.382773,-0.382773,-0.382773,-0.382773,-0.908714,-0.908714,...,-0.703704,-0.703704,-0.703704,-0.703704,0.345679,0.345679,0.345679,0.345679,0.085714,0.085714,0.085714,0.085714,-0.836145,-0.836145,-0.836145,-0.836145,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,1
3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.326531,0.326531,0.326531,0.326531,-0.891993,-0.891993,-0.891993,-0.891993,-0.203354,-0.203354,-0.203354,-0.203354,-0.219512,-0.219512,-0.219512,-0.219512,-0.828421,-0.828421,-0.828421,-0.828421,-0.729239,-0.729239,-0.729239,-0.729239,-0.836100,-0.836100,...,-0.777778,-0.777778,-0.777778,-0.777778,0.580247,0.580247,0.580247,0.580247,0.200000,0.200000,0.200000,0.200000,-0.937349,-0.937349,-0.937349,-0.937349,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,0
4,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.851024,-0.851024,-0.851024,-0.851024,0.358491,0.358491,0.358491,0.358491,0.304878,0.304878,0.304878,0.304878,1.000000,1.000000,1.000000,1.000000,-0.702202,-0.702202,-0.702202,-0.702202,-0.641079,-0.641079,...,-0.592593,-0.592593,-0.592593,-0.592593,0.345679,0.345679,0.345679,0.345679,0.142857,0.142857,0.142857,0.142857,-0.903614,-0.903614,-0.903614,-0.903614,0.333333,-0.153846,0.160377,-0.59322

# Criação de variáveis

In [ ]:
features_continuas = dados_limpos.columns[10:-1].tolist()
features_categoricas = dados_limpos.columns[0:9].tolist()

In [ ]:
X = dados_limpos.drop('ICU', axis = 1)
y = dados_limpos['ICU']

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X,y, test_size = 0.25, stratify = y, random_state = 527435)

# MLP

In [ ]:
modelo_mlp = MLPClassifier(random_state=527435)

In [ ]:
parametros_mlp = {'activation' : ['identity', 'logistic', 'tanh', 'relu'],
              'solver' : ['lbfgs', 'sgd', 'adam'],
              'learning_rate' : ['constant', 'invscaling', 'adaptive'],
              'max_iter' : [100, 200, 500],
              'tol' : [0.0001, 0.001, 0.01, 0.1,1]
    
}

In [ ]:
clf_mlp = GridSearchCV(modelo_mlp,parametros_mlp, scoring= 'roc_auc')

In [ ]:
clf_mlp.fit(X_treino,y_treino)

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_st...
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                        

In [ ]:
melhor_modelo_mlp = clf_mlp.best_estimator_
melhor_modelo_mlp

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=100,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=527435, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
clf_mlp.best_params_

{'activation': 'relu',
 'learning_rate': 'constant',
 'max_iter': 100,
 'solver': 'adam',
 'tol': 0.0001}

Melhores parâmetros:

{'activation': 'relu',
 'learning_rate': 'constant',
 'max_iter': 100,
 'solver': 'adam',
 'tol': 0.0001}

In [ ]:
clf_mlp.best_score_

0.7960316091954022

In [ ]:
y_pred_mlp = clf_mlp.predict(X_teste)

In [ ]:
print(classification_report(y_teste,y_pred_mlp))

              precision    recall  f1-score   support

           0       0.70      0.68      0.69        47
           1       0.64      0.66      0.65        41

    accuracy                           0.67        88
   macro avg       0.67      0.67      0.67        88
weighted avg       0.67      0.67      0.67        88



In [ ]:
modelo_mlp.fit(X_treino, y_treino)
y_pred_mlp = modelo_mlp.predict(X_teste)
acur = modelo_mlp.score(X_teste, y_teste)
print(f"Acurácia = {round(100*acur, 2)}%")

Acurácia = 69.32%


In [ ]:
print(classification_report(y_teste,y_pred_mlp))

              precision    recall  f1-score   support

           0       0.75      0.64      0.69        47
           1       0.65      0.76      0.70        41

    accuracy                           0.69        88
   macro avg       0.70      0.70      0.69        88
weighted avg       0.70      0.69      0.69        88



In [ ]:
print(classification_report(y_treino,modelo_mlp.predict(X_treino)))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       142
           1       0.89      0.85      0.87       121

    accuracy                           0.88       263
   macro avg       0.88      0.88      0.88       263
weighted avg       0.88      0.88      0.88       263



In [ ]:
import itertools
df_scores_mlp = pd.DataFrame(columns=['Repetições', 'Folds', 'Tolerância', 'Acurácia', 'Precisão', 'Revocação', 'F1'])
T = [10*x for x in range(0,11)]
R = [10,50,100]
F = [5,7,10]
ind = 0
for t,r,f in itertools.product(T,R,F):
  mlp_previsao, mlp_scores = modelo_repetido(melhor_modelo_mlp,
                            X_train =  X_treino, y_train =  y_treino,
                             X_test =  X_teste, y_test =  y_teste,
                             n_rep = r,n_folds = f, tol= t, verbosity = 0,
                             tol_overfit = 1)
  if mlp_scores == False:
    pass
  else:
    ac = mlp_scores['Acuracia'][0]
    pr = mlp_scores['Precisão'][0]
    rv = mlp_scores['Revocação'][0]
    f1 = mlp_scores['F1'][0]
    nova_coluna = [r,f,t,ac,pr,rv,f1]
    df_scores_mlp.loc[ind] = nova_coluna
    ind = ind+1
  
df_scores_mlp

,Repetições,Folds,Tolerância,Acurácia,Precisão,Revocação,F1
0,10.0,5.0,0.0,65.91,60.78,75.61,67.39
1,10.0,7.0,0.0,67.05,61.11,80.49,69.47
2,10.0,10.0,0.0,68.18,62.26,80.49,70.21
3,50.0,5.0,0.0,62.50,56.45,85.37,67.96
4,50.0,7.0,0.0,69.32,62.07,87.80,72.73
...,...,...,...,...,...,...,...
94,50.0,7.0,100.0,65.91,61.70,70.73,65.91
95,50.0,10.0,100.0,67.05,62.50,73.17,67.42
96,100.0,5.0,100.0,65.91,61.70,70.73,65.91
97,100.0,7.0,100.0,65.91,61.22,73.17,66.67


In [ ]:
df_scores_mlp.sort_values(by = 'Acurácia', ascending = False)

,Repetições,Folds,Tolerância,Acurácia,Precisão,Revocação,F1
92,10.0,10.0,100.0,70.45,70.27,63.41,66.67
83,10.0,10.0,90.0,70.45,70.27,63.41,66.67
36,10.0,5.0,40.0,70.45,72.73,58.54,64.86
65,10.0,10.0,70.0,69.32,67.50,65.85,66.67
16,100.0,7.0,10.0,69.32,62.96,82.93,71.58
...,...,...,...,...,...,...,...
90,10.0,5.0,100.0,53.41,0.00,0.00,0.00
81,10.0,5.0,90.0,53.41,0.00,0.00,0.00
63,10.0,5.0,70.0,53.41,0.00,0.00,0.00
82,10.0,7.0,90.0,53.41,0.00,0.00,0.00


Melhores parâmetros

Repetições = 10
Folds = 10
Tolerância = 100